# Spec

### This Is the Data You’re Looking For

In [1]:
;; A simple map

{:title "Getting Clojure" :author "Olsen" :copies 1000000}

{:title "Getting Clojure", :author "Olsen", :copies 1000000}

In [2]:
;; A function to check the shape pf the last dict

(defn book? [x]
    (and
        (map? x)
        (string? (:author x))
        (string? (:title x))
        (pos-int? (:copies x))))


#'user/book?

In [8]:
;; But for data validation, a better option is clojure.spec

(ns inventory.core 
    (:require [clojure.spec.alpha :as s]))


nil

In [9]:
;; The key function supplied by spec is valid?:

(s/valid? number? 44)


true

In [10]:
(s/valid? number? :hello)


false

In [11]:
;; You can combine predicates to check if the value is a number 
;; and greater than 10 with 'and', as follows

(def n-gt-10 (s/and number? #(> % 10)))

(s/valid? n-gt-10 1)


false

In [13]:
(s/valid? n-gt-10 10)


false

In [15]:
(s/valid? n-gt-10 11)


true

In [16]:
;; 'and' isn't limitted to just two predicates

(def n-gt-10-lt-100
    (s/and number? #(> % 10) #(< % 100)))


#'inventory.core/n-gt-10-lt-100

In [18]:
;; To match either a number or a string, we can use 'or' as follows

(def n-or-s (s/or :a-number number? :a-string string?))

(s/valid? n-or-s "Hello!")


true

In [19]:
(s/valid? n-or-s 99)


true

In [20]:
(s/valid? n-or-s 'foo)


false

In [21]:
;; This defines a spec that will accept numbers greater than 10, or any symbol

(def n-gt-10-or-s 
    (s/or :greater-10 n-gt-10 :a-symbol symbol?)) ; Specs can be used inside other specs


#'inventory.core/n-gt-10-or-s

### Spec’ing Collections

In [ ]:
;; Check if it's a collection of strings

(def coll-of-strings (s/coll-of string?))


In [23]:
;; Check if it's a collection of numbers or strings

(def coll-of-n-or-s (s/coll-of n-or-s))

#'inventory.core/coll-of-n-or-s

In [24]:
;; To match only four-element collections consisting of alternating 
;; strings and numbers, we can use 'cat' as follows

(def s-n-s-n 
    (s/cat :s1 string? :n1 number? :s2 string? :n2 number?))

(s/valid? s-n-s-n ["Emma" 1815 "Jaws" 1974])


true

In [28]:
; Specs on maps can also be checked with 'keys', as follows

(def book-s
    (s/keys :req-un [:inventory.core/title
                     :inventory.core/author
                     :inventory.core/copies]))

(s/valid? book-s {:title "Emma" :author "Austen" :copies 10})


true

In [29]:
(s/valid? book-s {:title "Arabian Nights" :copies 17})


false

In [30]:
;; Additional entries are OK

(s/valid? book-s {:title "2001" :author "Clarke" :copies 1 :published 1968}) 


true

### Registering Specs


In [31]:
;; You can register your spec in a JVM-wide registry of specs with 'def', as follows

(s/def :inventory.core/book
    (s/keys :req-un [:inventory.core/title
                     :inventory.core/author
                     :inventory.core/copies]))


:inventory.core/book

In [33]:
;; Once a spec is registered, you can use the keyword as a spec, as follows

(s/valid? :inventory.core/book 
    {:title "Dracula" :author "Stoker" :copies 10})


true

In [35]:
;; But if the current namespace is inventory.core, the previous example
;; can be modified as follows

(s/def ::book 
    (s/keys :req-un [::title ::author ::copies]))

(s/valid? ::book 
    {:title "Dracula" :author "Stoker" :copies 10})


true

### Spec’ing Maps (Again)


In [36]:
;; In the last snippet, we onle checked that the 'book' map has certain keys.
;; To check the values themselves, we can add them to the spec, as follows

(s/def ::title string?)

(s/def ::author string?)

(s/def ::copies int?)


:inventory.core/copies

In [41]:
;; This check will fail

(s/valid? ::book {:author :austen :title :emma})


false

In [39]:
;; To see an explanation of the failure, you can use 'explain' as follows

(s/explain ::book {:author :austen :title :emma})


:austen - failed: string? in: [:author] at: [:author] spec: :inventory.core/author
:emma - failed: string? in: [:title] at: [:title] spec: :inventory.core/title
{:author :austen, :title :emma} - failed: (contains? % :copies) spec: :inventory.core/book


nil

In [44]:
;; To see what a successful check will look, you can use 'conform' and a target spec, as follows

(s/conform s-n-s-n ["Emma" 1815 "Jaws" 1974])


{:s1 "Emma", :n1 1815, :s2 "Jaws", :n2 1974}

### Function Specs


In [47]:
;; Assert that an inventory is a collection of books.

(s/def :inventory.core/inventory
    (s/coll-of ::book))


:inventory.core/inventory

In [48]:
;; You can add a spec in the pre or post-conditions of a function, as follows

(defn find-by-title
    [title inventory]
    {:pre [(s/valid? ::title title)
           (s/valid? ::inventory inventory)]}
    (some #(when (= (:title %) title) %) inventory))


#'inventory.core/find-by-title

In [50]:
;; But a better way is to separate the function and the spec, as follows

;; The find-by-title function.

(defn find-by-title [title inventory]
    (some #(when (= (:title %) title) %) inventory))

;; The spec for the find-by-title function.

(s/fdef find-by-title
    :args (s/cat :title ::title
                 :inventory ::inventory))


inventory.core/find-by-title

In [51]:
;; To apply the argument checking, we need another namespace as follows

(require '[clojure.spec.test.alpha :as st])

(st/instrument 'inventory.core/find-by-title) ; Explicitly call the function


[inventory.core/find-by-title]

In [57]:
;; Calling the function with wrong args will throw an exception
;; with a description of the reasons

(find-by-title "Emma" ["Emma" "2001" "Jaws"]) ; (*)

; For performance reasons, enable specs in development/testing


Execution error - invalid arguments to inventory.core/find-by-title at (REPL:4).
"Emma" - failed: map? at: [:inventory] spec: :inventory.core/book
"2001" - failed: map? at: [:inventory] spec: :inventory.core/book
"Jaws" - failed: map? at: [:inventory] spec: :inventory.core/book


class clojure.lang.ExceptionInfo: 

### Spec-Driven Tests


In [58]:
;; A simple function with its spec

(defn book-blurb [book]
    (str "The best selling book " (:title book) " by " (:author book)))

(s/fdef book-blurb :args (s/cat :book ::book))


inventory.core/book-blurb

In [60]:
;; Now we can run the function with 1000 random input data, as follows

(require '[clojure.spec.test.alpha :as stest])

(stest/check 'inventory.core/book-blurb)


({:spec #object[clojure.spec.alpha$fspec_impl$reify__2524 0x62acd031 "clojure.spec.alpha$fspec_impl$reify__2524@62acd031"], :clojure.spec.test.check/ret {:result true, :pass? true, :num-tests 1000, :time-elapsed-ms 599, :seed 1646221360039}, :sym inventory.core/book-blurb})

In [61]:
;; We can also add a check for the return value, as follows

(s/fdef book-blurb
    :args (s/cat :book ::book)
    :ret (s/and string? (partial re-find #"The best selling")))


inventory.core/book-blurb

In [65]:
;; We can add additional checks using external functions, as follows

(defn check-return [{:keys [args ret]}]
    (let [author (-> args :book :author)]
    (not (neg? (.indexOf ret author)))))

(s/fdef book-blurb
    :args (s/cat :book ::book)
    :ret (s/and string? (partial re-find #"The best selling"))
    :fn check-return)


inventory.core/book-blurb

Issues with clojure.spec

In [67]:
;; Be careful with mistyping keys

(s/def ::author string?)

(s/def ::titlo string?) ; Should be ::title

(s/def ::copies pos-int?)

(s/def ::book
    (s/keys :req-un [::title ::author ::copies]))

;; Register a spec for the find-by-title function.

(s/fdef find-by-title
    :args (s/cat :title ::title
    :inventory ::inventory))


inventory.core/find-by-title